# colab

In [2]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/bias gpt")
import os
os.environ['WANDB_DISABLED'] = 'true'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install datasets

# Head

In [ ]:
import torch, importlib
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import GPT2TokenizerFast, DistilBertTokenizerFast, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import matplotlib.pyplot as plt
from datasets import load_dataset
from torch.utils.data import random_split

model_name = "distilbert-base-uncased"
bert_tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

model_name = "distilgpt2"
gpt_tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


# 原神

In [3]:
prompt = "\n"
sentence = prompt + open('Genshin Impact.txt').read().replace('\n', ' ')
bert_inputs = bert_tokenizer(sentence, return_tensors="pt", max_length = 128, truncation = True,
                             add_special_tokens = False)
gpt_inputs = gpt_tokenizer(sentence, return_tensors="pt", max_length = 128, truncation = True)

NameError: name 'bert_tokenizer' is not defined

In [ ]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()
tvae.train()
epochs = 100
optim = AdamW(tvae.parameters(), 5e-4)
scheduler = CosineAnnealingLR(optim, T_max = epochs, eta_min = 0)
for epoch in range(epochs):
    loss = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'])[0]
    loss.backward()
    optim.step()
    optim.zero_grad()
    scheduler.step()
    print(f"epoch: {epoch + 1}; loss: {loss.item()}")

In [ ]:
logits = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = False)[2]
print(gpt_tokenizer.decode(logits.argmax(-1)[0]))
print((logits.argmax(-1)[0] != gpt_inputs['input_ids'][0, 1:]).sum().item())

In [ ]:
tvae.eval()
bias = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = True)[1]
print(bias.shape)
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2')
if torch.cuda.is_available():
    gpt = gpt.cuda()
with torch.no_grad():
    gpt.transformer.h[-1].mlp.c_fc.bias += bias[0]

In [ ]:
with torch.no_grad():
    tokens = gpt(gpt_inputs['input_ids'])[0][0].argmax(-1)
print(gpt_tokenizer.decode(tokens))
print((tokens[:-1] != gpt_inputs['input_ids'][0, 1:]).sum().item())

In [ ]:
test_ids = gpt_tokenizer(prompt, return_tensors="pt").input_ids
if torch.cuda.is_available():
   test_ids = test_ids.to('cuda')

generated_ids = gpt.generate(
    test_ids,
    max_length = 128,
    num_return_sequences = 1,
    no_repeat_ngram_size = 5,
    num_beams = 5,
    eos_token_id = 50256,
    bad_words_ids = [[198]],
)

# 解码生成的文本
generated_text = gpt_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text[0])
print(len(generated_ids[0]))

# wiki

In [ ]:
wiki = load_dataset('wikipedia', '20220301.simple',
          trust_remote_code = True)["train"].select(range(1000))

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/205328 [00:00<?, ? examples/s]

In [ ]:
def mapping(x):
  bert = bert_tokenizer(x['text'], return_tensors="pt", padding = 'max_length',
                        max_length = 128, truncation = True, add_special_tokens = False)
  gpt_text = ['\n' + text.replace('\n',' ') for text in x['text']]
  gpt = gpt_tokenizer(gpt_text, return_tensors="pt", padding = 'max_length',
                      max_length = 128, truncation = True)
  return{
    'bert_ids': bert['input_ids'],
    'bert_mask': bert['attention_mask'],
    'gpt_ids': gpt['input_ids'],
    'gpt_mask': gpt['attention_mask'],
  }

dataset = wiki.map(mapping, batched = True, load_from_cache_file = True,
                   remove_columns=['id','url','title','text'])
dataset.set_format('torch')
print(dataset)
train_size = len(dataset) - len(dataset) // 20
trainData, evalData = random_split(dataset, [train_size, len(dataset) // 20])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['bert_ids', 'bert_mask', 'gpt_ids', 'gpt_mask'],
    num_rows: 1000
})


In [ ]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()

In [ ]:
gpt_tokenizer.batch_decode(tvae(**trainData[0:3])[2].argmax(-1))

0.6510096788406372 0.0


['European (: official name: United Mexican States ) official is a country in North America. Mexico is a of Mexico, Texas and other American states. Mexico and Belize are south of Texas. Guatemala is south Mexico Pacific Ocean and the Gulf of Mexico. Mexico People living in Mexico or who are from there are called Mexicans.  Mexican people speak Mexican.  are also Mexicans who speak Spanish American languages. like Nativeahuatl, Nan, and Zapotec.  capital of Mexico is Mexico City.  People  Before the Europeans came, many Native American cultures existed in Mexico. The earliest was the Olmec culture in',
 "Andorra, officially the Principality of Andorra (), is a landlocked sovereign country located in the eastern Pyrenees Mountains of Southern Europe. bordered by Spain and France. It are about 10,000 people living in the country. There capital is Andorra la Vella. There is ruled by a Spanish Bishop and a French President. Theorra is government is a parliamentary democracy. And Thereorra'

In [ ]:
training_args = TrainingArguments(
    output_dir = "./results",
    save_strategy = "no",
    num_train_epochs = 100,
    weight_decay = 0,
    learning_rate = 5e-4,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    warmup_steps = 10,
    logging_steps = 32,
    # eval_strategy= "steps",
    # eval_steps = 10,
    fp16 = True,
    lr_scheduler_type = 'linear',
    max_grad_norm  = 1,
)

trainer = Trainer(
  model = tvae,
  args = training_args,
  train_dataset = trainData,
  # eval_dataset = evalData,
)
trainer.train()

In [ ]:
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2').to('cuda')
input_ids = torch.tensor([[1,2,3]])
mask = torch.tensor([[1,1,0]])
gpt(input_ids, attention_mask = mask)[0] * mask.unsqueeze(-1)

In [ ]:
!python TextVAE.py

2024-11-16 06:29:34.124057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 06:29:34.143725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 06:29:34.149694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 06:29:35.414994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
0.0


# TF_TextVAE

In [9]:
!python TF_trainer.py --size 10 --batch_size 32 --warmup 10 --epoch 100

2024-11-19 06:59:06.376938: I tensorflow/core/tpu/tpu_api_dlsym_initializer.cc:95] Opening library: /usr/local/lib/python3.10/dist-packages/tensorflow/python/platform/../../libtensorflow_cc.so.2
2024-11-19 06:59:06.377124: I tensorflow/core/tpu/tpu_api_dlsym_initializer.cc:119] Libtpu path is: libtpu.so
2024-11-19 06:59:06.428990: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/descriptor_database.cc:669] Symbol name "xla.cpu.OneDnnMatMulConfig" conflicts with the existing symbol "xla.cpu.OneDnnMatMulConfig".
[libprotobuf FATAL external/com_google_protobuf/src/google/protobuf/descriptor.cc:1986] CHECK failed: GeneratedDatabase()->Add(encoded_file_descriptor, size): 
terminate cal

In [ ]:
from TF_TextVAE import TextVAE

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import AdamW # type: ignore
from transformers import GPT2TokenizerFast
from datasets import load_dataset
from TF_TextVAE import TextVAE
import argparse, math


# 环境设置
# TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [38]:
import tensorflow as tf
prompt = "\n"
sentence = prompt + open('Genshin Impact.txt').read().replace('\n', ' ')
inputs = gpt_tokenizer(sentence, return_tensors="tf", max_length = 128, truncation = True)
inputs['attention_mask'] = tf.cast(inputs['attention_mask'], tf.float32)

In [39]:
import TF_TextVAE
importlib.reload(TF_TextVAE)
from TF_TextVAE import TextVAE
from tensorflow.keras.optimizers import AdamW

tvae = TextVAE()
tvae.compile(
    optimizer = AdamW(5e-4, weight_decay = 0, clipnorm = 1.0),
    loss = lambda y_true, y_pred: y_pred
  )

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.
All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [40]:
def generator():
  yield (inputs, tf.convert_to_tensor([1], dtype = tf.int32))
tf_dataset = tf.data.Dataset.from_generator(
  generator, output_signature=({
    'input_ids': tf.TensorSpec(shape=(None,), dtype=tf.int32),
    'attention_mask': tf.TensorSpec(shape=(None,), dtype=tf.float32),
  }, tf.TensorSpec(shape=(None,), dtype=tf.int32))
)
tf_dataset

<_FlatMapDataset element_spec=({'input_ids': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [49]:
tvae(inputs = inputs)

AttributeError: 'TextVAE' object has no attribute 'eval'